# $\pi^0$ PRD

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#pi0scaling = {"val": 0, "suffix": "", "title": "No pi0 Scaling"}
pi0scaling = {"val": 1, "suffix": "_pi0flat", "title": "0.759 pi0 Scaling"}

In [ ]:
from matplotlib import pyplot as plt

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
plt.rcParams.update(params)

from load_data_run123 import *
from unblinding_far_sideband import *

In [ ]:
import sys
import localSettings as ls
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
import scipy.stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
which_sideband = ['fulldata'     ,'fulldata'] 

In [ ]:
samples = load_data_run123(which_sideband=which_sideband[0],
                           return_plotter=False,
                           pi0scaling=pi0scaling["val"],
                           loadpi0variables=True,
                           loadtruthfilters=True,
                           loadshowervariables=True,
                           loadnumuntuples=False,
                           loadnumuvariables=False,
                           loadfakedata=0,
                           USEBDT=True,
                           loadsystematics=True,
                           loadrecoveryvars=False)

In [ ]:
import plotter
import unblinding_far_sideband
import importlib
importlib.reload(plotter)
importlib.reload(unblinding_far_sideband)

In [ ]:
plotter_runbyrun = {}
for run in [1, 2, 3, 123, 12]:
    this_weights, this_pot = get_weights(run,dataset=which_sideband[1])
    plotter_runbyrun[run] = plotter.Plotter(samples, this_weights, pot=this_pot)

# $\pi^0$ Selection

In [ ]:
from unblinding_far_sideband import *

sdb_key = 'TwoPShr'
pre_key = 'NUE'
sel_key = 'PI0SEL'
run = 123

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]
sel =  selection_categories[sel_key]

DETSYS = {'mc':True,'nue':True,'ccpi0':True,'ncpi0':True,'dirt':True,\
          'ncnopi':True,'nccpi':True,'ncpi0':True,'ccpi0':True}

DETSYSPATH = ls.ntuple_path+'/detsys/{}_{}_{}/'.format(sideband['dir'], preselection['dir'], sel['dir'])

QUERY = ""
for query in [sideband['query'],preselection['query'],sel['query']]:
    if query is None: continue
    if QUERY: QUERY += ' and '
    QUERY += query
    if run in [1, 2, 3, 12]:
        QUERY += f' and run{run}==True'
        
SELQUERY = '{}_{}_{}_'.format(sideband['dir'], preselection['dir'], sel['dir'])        

TITLE = ""
for text in [f'Run {run}',' - ',sideband['title'],' - ',preselection['title'],'\n',sel['title'],' - ',pi0scaling["title"]]:
    if text is None: continue
    TITLE += text

# Fig. 7: $\pi^0$ Mass Peak

In [ ]:
VARIABLE, BINS, RANGE, XTIT = 'pi0_mass_Y_corr',19,(10,200),r"$M_{\gamma\gamma}$ [MeV/$c^2$]"

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARIABLE,   
    query=QUERY,
    figtitle="MicroBooNE",
    labeldecimals=0,
    predictedevents=False,
    kind="paper_category",
    draw_sys=True,
    detsysdict=DETSYS,
    DETSYSPATH=DETSYSPATH,
    COVMATRIX="",
    ratio=False,
    stacksort=3,
    title=XTIT,
    bins=BINS,
    range=RANGE,
    chisq=False,
)[0:3]

TITLE = r'$\pi^0$ selection'
ax1.set_title(TITLE, loc='left')
ax1.set_ylim(0., ax1.get_ylim()[1]*1.1)

print ('prediction : ', plotter_runbyrun[run].prediction)
print ('mc stat diag entries : ', np.sqrt(np.diagonal(plotter_runbyrun[run].cov_mc_stat)))
print ('data : ',plotter_runbyrun[run].data)
print ('EXT : ',plotter_runbyrun[run].ext)
print ('EXT errors : ',plotter_runbyrun[run].ext_err)

plt.tight_layout()
fig.show() 
save_path = ls.plots_path
#fig.savefig('/Users/davidc-local/Neutrinos/NUE/PRD/'+SELQUERY+VARIABLE+"_PRD.pdf") 

# $\pi^0$ Mass Fit

In [ ]:
import scipy.optimize
from scipy.optimize import curve_fit

def gauss(x,A,mu,sigma):
    norm = A/(np.sqrt(2*np.pi)*sigma)
    exp  = np.exp(-((x-mu)**2)/(2*sigma*sigma))
    return norm * exp

def gauss_exp(x, n, mu, sigma, k):
    sigma = abs(sigma)
    condition = (x - mu) / sigma >= -k    
    y = np.copy(x)
    y[condition] = n * np.exp(-0.5 * ((x[condition] - mu) / sigma)**2)
    y[~condition] = n * np.exp(k**2 / 2 + k * ((x[~condition] - mu) / sigma))
#     print(x)
    return y

def get_function_max(f, *args):
    def func(x, *arg):
        return -f(x, *arg)
    return f(scipy.optimize.fmin(func, 0, args=args, disp=False)[0], *args)

def find_nearest(array, value):
    array = np.asarray(array)
    #print (array)
    idx = (np.abs(array - value)).argmin()
    return idx

def mpv(array):
    if sum(array) < 5:
        return np.median(array)
    
    n_bins = energy_bins
    r = energy_range
    hist, bin_edges = np.histogram(array, bins=n_bins, range=r)
    
    bin_centers = [i*r[1]/n_bins-r[1]/(n_bins*2) for i in range(1,n_bins+1)]
    try:
        popt, pcov = scipy.optimize.curve_fit(gauss_exp, bin_centers, hist, maxfev=10000)
        return scipy.optimize.fmin(lambda x: -gauss_exp(x, *popt), 0)
    except RuntimeError:
        return np.median(array)
    
def fwhm(array):
    if sum(array) < 5:
        return np.std(array)

    n_bins = energy_bins
    r = energy_range
    hist, bin_edges = np.histogram(array, bins=n_bins, range=r)
    
    bin_centers = [i*r[1]/n_bins-r[1]/(n_bins*2) for i in range(1,n_bins+1)]

    try:
        popt, pcov = scipy.optimize.curve_fit(gauss_exp, bin_centers, hist, maxfev=10000)
        x_values = np.linspace(r[0], r[1], 1000)
        y_values = gauss_exp(np.linspace(r[0], r[1], 1000), *popt)
        try:
            x_max = scipy.optimize.fmin(lambda x: -gauss_exp(x, *popt), 0)
        except RuntimeError:
            x_max = np.median(array)
        y_max = find_nearest(y_values, gauss_exp(x_max, *popt))
        y_max_value = y_values[y_max]
        fwhm1 = find_nearest(y_values[:y_max], y_max_value/2)
        fwhm2 = find_nearest(y_values[y_max:], y_max_value/2)
        x_2 = x_values[y_max:][fwhm2]     
        x_1 = x_values[:y_max][fwhm1]
        return x_2-x_1
    except RuntimeError:
        return np.std(array)

In [ ]:
BKGD = np.array([ 1.257,4.392,4.372,6.693,9.447,9.73,15.539,19.589,21.453,\
         27.607,30.9,24.214,24.358,22.274,19.015,15.504,14.429,12.097,8.851])

DATA = np.array([1,8,13,20,31,44,57,85,101,120,117,137,109,92,54,33,40,\
        22,22])

MC = np.array([2.22,7.375,9.468,18.034,28.22,38.831,55.987,79.923,\
         100.817,111.843,127.89,121.022,105.664,83.88,58.787,45.696,32.446,\
         24.482,17.312])


MCSGNL = MC - BKGD
DATASGNL = DATA - BKGD

DATAERR = np.sqrt(DATASGNL)
MCERR = np.sqrt(MCSGNL)


In [ ]:
fig = plt.figure(figsize=(6,6))
#BINC = np.array([0.3,0.4,0.5,0.6,0.7])


# pi0s
BINC = np.linspace(10,200,19) + 5
# gauss
#guess = [155,135.,30]
#BOUNDS = ([120,100,10],[2e4,150,50])
# gauss_exp
guess = [155,135.,30,0.8]
BOUNDS = ([40,100,10,0.5],[2e4,150,50,0.9])

print ('bin centers: ',BINC)

BINMIN = 2
BINMAX = 18

# MC

plt.errorbar(BINC,DATASGNL,xerr=10,yerr=MCERR,fmt='o',color='r')

popt,popv = curve_fit(gauss_exp,BINC[BINMIN:BINMAX],MCSGNL[BINMIN:BINMAX],\
                      p0=guess,\
                      #method='dogbox',\
                      sigma=MCERR[BINMIN:BINMAX],absolute_sigma=True,\
                     bounds=BOUNDS)
perr = np.sqrt(np.diag(popv))
print ('popt : ',popt)

# gauss_exp
mass = popt[1]
width = popt[2]
masserr = perr[1]
# gauss
#mass = popt[0]
#width = popt[1]
#masserr = perr[0]

masserr = width / np.sqrt(np.sum(MCSGNL))

xvals = np.linspace(0.,200.,100)
#xvals = np.linspace(250.,750.,100)
plt.plot(xvals,gauss_exp(xvals,*popt),'r--',lw=2,\
         label='$\mu$ = %.0f $\pm$ %.0f MeV \n $\sigma$ = %.0f MeV'%(mass,masserr,width))

# DATA

plt.errorbar(BINC,MCSGNL,xerr=10,yerr=MCERR,fmt='o',color='b')

popt,popv = curve_fit(gauss_exp,BINC[BINMIN:BINMAX],DATASGNL[BINMIN:BINMAX],\
                      p0=guess,\
                      #method='dogbox',\
                      sigma=DATAERR[BINMIN:BINMAX],absolute_sigma=True,\
                     bounds=BOUNDS)
perr = np.sqrt(np.diag(popv))
print ('popt : ',popt)

# gauss_exp
mass = popt[1]
width = popt[2]
masserr = perr[1]
# gauss
#mass = popt[0]
#width = popt[1]
#masserr = perr[0]

masserr = width / np.sqrt(np.sum(DATASGNL))

xvals = np.linspace(0.,200.,100)
#xvals = np.linspace(250.,750.,100)
plt.plot(xvals,gauss_exp(xvals,*popt),'b--',lw=2,\
         label='$\mu$ = %.0f $\pm$ %.0f MeV \n $\sigma$ = %.0f MeV'%(mass,masserr,width))


plt.legend(loc=2)
#plt.ylim([0,18])
plt.xlabel(r'$M_{\gamma\gamma}$ [MeV]',\
           fontsize=18)
plt.ylabel('background subtracted events',fontsize=18)
plt.tight_layout()
plt.show()
save_path = '/Users/davidc-local/Neutrinos/ETA/TN/reconstruction/'
fig.savefig(save_path + "etamassfit" + '_v2.pdf', dpi=250)  